In [5]:
%reload_ext autoreload
%autoreload 2

from pytorch_lightning import seed_everything
from src.acnets.deep import WaveletModel, MaskedModel
from src.acnets.datasets import LEMONDataModule, Julia2018DataModule

for n_embeddings in [4, 8, 16, 32, 64, 128, 256]:

    seed_everything(42)
    feature_type = 'time_networks'

    pretrain_datamodule = LEMONDataModule(
        atlas='dosenbach2010', kind='partial correlation',
        aggregation_strategy=feature_type,
        test_ratio=.1, val_ratio=.05,
        segment_length=32,
        n_subjects=215, batch_size=32, shuffle=True)
    pretrain_datamodule.setup()

    finetune_datamodule = Julia2018DataModule(
        atlas='dosenbach2010', kind='partial correlation',
        aggregation_strategy='time_networks',
        segment_length=32,
        test_ratio=.5, batch_size=32, shuffle=True)
    finetune_datamodule.setup()

    n_features = finetune_datamodule.train[0][0].shape[-1]

    model = MaskedModel(n_features, n_embeddings)

    trainer = model.fit(pretrain_datamodule, max_epochs=2, phase='pretrain')
    tuner = model.fit(finetune_datamodule, max_epochs=2, phase='finetune')

Seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


ckpt_id: 2206f7


/home/morteza/miniforge3/envs/acnets/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/checkpoint_connector.py:186: .fit(ckpt_path="last") is set, but there is no last checkpoint available. No checkpoint will be loaded.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name              | Type               | Params
---------------------------------------------------------
0 | accuracy          | MulticlassAccuracy | 0     
1 | feature_extractor | Seq2SeqAE          | 540   
2 | cls_head          | Classifier         | 36    
---------------------------------------------------------
576       Trainable params
0         Non-trainable params
576       Total params
0.002     Total estimated model params size (MB)
`Trainer.fit` stopped: `max_epochs=2` reached.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name              | T

ckpt_id: 7d1ff2


/home/morteza/miniforge3/envs/acnets/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/checkpoint_connector.py:186: .fit(ckpt_path="last") is set, but there is no last checkpoint available. No checkpoint will be loaded.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name              | Type               | Params
---------------------------------------------------------
0 | accuracy          | MulticlassAccuracy | 0     
1 | feature_extractor | Seq2SeqAE          | 1.0 K 
2 | cls_head          | Classifier         | 118   
---------------------------------------------------------
1.1 K     Trainable params
0         Non-trainable params
1.1 K     Total params
0.004     Total estimated model params size (MB)
`Trainer.fit` stopped: `max_epochs=2` reached.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name              | T

...

/home/morteza/miniforge3/envs/acnets/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/checkpoint_connector.py:186: .fit(ckpt_path="last") is set, but there is no last checkpoint available. No checkpoint will be loaded.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name              | Type               | Params
---------------------------------------------------------
0 | accuracy          | MulticlassAccuracy | 0     
1 | feature_extractor | Seq2SeqAE          | 20.9 K
2 | cls_head          | Classifier         | 6.3 K 
---------------------------------------------------------
27.2 K    Trainable params
0         Non-trainable params
27.2 K    Total params
0.109     Total estimated model params size (MB)
`Trainer.fit` stopped: `max_epochs=2` reached.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name              | T

ckpt_id: 3c0af2


/home/morteza/miniforge3/envs/acnets/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/checkpoint_connector.py:186: .fit(ckpt_path="last") is set, but there is no last checkpoint available. No checkpoint will be loaded.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name              | Type               | Params
---------------------------------------------------------
0 | accuracy          | MulticlassAccuracy | 0     
1 | feature_extractor | Seq2SeqAE          | 74.4 K
2 | cls_head          | Classifier         | 24.9 K
---------------------------------------------------------
99.3 K    Trainable params
0         Non-trainable params
99.3 K    Total params
0.397     Total estimated model params size (MB)
`Trainer.fit` stopped: `max_epochs=2` reached.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name              | T

ckpt_id: a1698e


/home/morteza/miniforge3/envs/acnets/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/checkpoint_connector.py:186: .fit(ckpt_path="last") is set, but there is no last checkpoint available. No checkpoint will be loaded.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name              | Type               | Params
---------------------------------------------------------
0 | accuracy          | MulticlassAccuracy | 0     
1 | feature_extractor | Seq2SeqAE          | 279 K 
2 | cls_head          | Classifier         | 98.9 K
---------------------------------------------------------
378 K     Trainable params
0         Non-trainable params
378 K     Total params
1.515     Total estimated model params size (MB)
`Trainer.fit` stopped: `max_epochs=2` reached.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name              | T